: 

# Project IV: Image Processing and Computer Vision

# Data loading 

CIFAR-10 
- consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. 
- There are 50000 training images and 10000 test images.

In [ ]:
import pickle
import numpy as np
import pandas as pd


# Function to unpickle CIFAR-10 data
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

# Load CIFAR-10 dataset
train1 = unpickle("/Users/tiagovhp/Ironhack/Week_6/Project/cifar-10-batches-py/data_batch_1")
train2 = unpickle("/Users/tiagovhp/Ironhack/Week_6/Project/cifar-10-batches-py/data_batch_2")
train3 = unpickle("/Users/tiagovhp/Ironhack/Week_6/Project/cifar-10-batches-py/data_batch_3")
train4 = unpickle("/Users/tiagovhp/Ironhack/Week_6/Project/cifar-10-batches-py/data_batch_4")
train5 = unpickle("/Users/tiagovhp/Ironhack/Week_6/Project/cifar-10-batches-py/data_batch_5")
test_data = unpickle("/Users/tiagovhp/Ironhack/Week_6/Project/cifar-10-batches-py/test_batch")
label_names = unpickle("/Users/tiagovhp/Ironhack/Week_6/Project/cifar-10-batches-py/batches.meta")



: 

# Visualization of labeled images


In [ ]:
# Function to reshape images
def reshape_image(image):
    # Separate into r,g,b channels
    red_channel = image[:1024]
    green_channel = image[1024:2048]
    blue_channel = image[2048:3072]
    # Reshape each channel into numpy array of each channel (32,32)
    red_channel = red_channel.reshape(32,32)
    green_channel = green_channel.reshape(32,32)
    blue_channel = blue_channel.reshape(32,32)
    # Stack the channels into a new axis to form a 3d array
    reshaped_array = np.stack((red_channel,green_channel,blue_channel), axis=2)
    return reshaped_array

# Function to return name of the image
def label_image(image_nr):
    #image = train1[b'data'][img_nr]
    label_code = train1[b'labels'][img_nr]
    label_name = label_names[b'label_names'][label_code]
    return label_name

# Reshape and label an image
img_nr = 10 # index of image (0 to 10000)
image = train1[b'data'][img_nr]  #original array of image
reshaped_image = reshape_image(image)
image_labeled = label_image(img_nr)

# Plot the image
import matplotlib.pyplot as plt
plt.imshow(reshaped_image)
plt.title(image_labeled)



: 

# Data Normalization

In [ ]:
# Normalize datasets
def normalize_data(data):
    data[b'data'] = data[b'data'] / 255.0

normalize_data(train1)
normalize_data(train2)
normalize_data(train3)
normalize_data(train4)
normalize_data(train5)
normalize_data(test_data)

: 

In [ ]:

# Function to reshape entire datasets
reshaped_data = np.zeros((10000,32,32,3))
def reshape_dataset(data):
    reshaped_data = np.zeros((10000,32,32,3))
    for i in range(len(data[b'data'])):
        reshaped_data[i]= reshape_image(data[b'data'][i]) #reshape image i
    return reshaped_data



: 

# One-Hot encoding of categorical variables

In [ ]:
# Function to one-hot encode labels of dataset
def labels_encoded(data):
    return pd.get_dummies(data[b'labels']).values




: 

In [ ]:
# Reshape and encode training data
train_images = np.vstack([reshape_dataset(train) for train in [train1, train2, train3, train4, train5]])
train_labels = np.vstack([labels_encoded(train) for train in [train1, train2, train3, train4, train5]])

: 

# Model Architecture - Transfer learning

In [ ]:
# Load the pre-trained model ResNet50
from tensorflow.keras.applications import EfficientNetB0

# Load ResNet50 pre-trained on ImageNet without the top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # we exclude the top fully connected layers

# Fine-tune: Unfreeze the top layers of the base model
base_model.trainable = True
# Freeze all layers except the last few
for layer in base_model.layers[:-20]:  # You can experiment with the number of layers to unfreeze
    layer.trainable = False



: 

In [ ]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

# Define the model
model = Sequential()
# Add the pre-trained base model
model.add(base_model)
# Add the remaining part of my custom architecture
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu')),
model.add(layers.BatchNormalization()),
model.add(layers.Dropout(0.5)),  # Dropout for regularization
model.add(llayers.Dense(10, activation='softmax'))  # CIFAR-10 has 10 classes

: 

# Compile the model

In [ ]:
# Compile the model with SGD optimizer
optimizer = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

: 

: 

# Train the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Set up callbacks for learning rate adjustment and early stopping
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss',  
                               patience=5,          
                               restore_best_weights=True)

# Train the model with batch_train_1
model.fit(train_images, train_labels, epochs=50, batch_size=64, validation_split=0.2, callbacks=[lr_scheduler, early_stopping])

: 

# Evaluate the model on test set

In [ ]:
# Evaluate the model on the test set
test_images = reshape_dataset(test_data)
test_labels = labels_encoded(test_data)

loss, accuracy = model.evaluate(test_images, test_labels)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


: 

# Make prediction on test set

In [ ]:
# Make predictions
pred_prob = model.predict(test_images)
# Get the predicted class by choosing the index with the highest probability\
y_pred = np.argmax(pred_prob, axis=1)

: 